In [44]:
import numpy as np
import pandas as pd
import csv
import nltk
from nltk.corpus import wordnet as wn

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marlo\AppData\Roaming\nltk_data...


True

### 1. Load National Research Council

In [38]:
# lexicon = pd.read_csv("NRC-Emotion-Lexicon/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt", sep='\t', )
file_path = "NRC-Emotion-Lexicon/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
lexicon = {}
with open(file_path, newline='') as csvfile:
    text = csv.reader(csvfile, delimiter='\t', quotechar='|')
    for row in text:
        if row[0] not in lexicon.keys():
            lexicon[row[0]] = {row[1]: row[2]}
        else:
            lexicon[row[0]][row[1]] = row[2]

In [39]:
# Vista primors 5 elementos del diccionario
for k,v in list(lexicon.items())[:5]:
    print(k, v)

aback {'anger': '0', 'anticipation': '0', 'disgust': '0', 'fear': '0', 'joy': '0', 'negative': '0', 'positive': '0', 'sadness': '0', 'surprise': '0', 'trust': '0'}
abacus {'anger': '0', 'anticipation': '0', 'disgust': '0', 'fear': '0', 'joy': '0', 'negative': '0', 'positive': '0', 'sadness': '0', 'surprise': '0', 'trust': '1'}
abandon {'anger': '0', 'anticipation': '0', 'disgust': '0', 'fear': '1', 'joy': '0', 'negative': '1', 'positive': '0', 'sadness': '1', 'surprise': '0', 'trust': '0'}
abandoned {'anger': '1', 'anticipation': '0', 'disgust': '0', 'fear': '1', 'joy': '0', 'negative': '1', 'positive': '0', 'sadness': '1', 'surprise': '0', 'trust': '0'}
abandonment {'anger': '1', 'anticipation': '0', 'disgust': '0', 'fear': '1', 'joy': '0', 'negative': '1', 'positive': '0', 'sadness': '1', 'surprise': '1', 'trust': '0'}


### 2. Extender el léxico NRC utilizando WordNet

In [51]:
wn.synsets('abandoned')

[Synset('abandon.v.01'),
 Synset('abandon.v.02'),
 Synset('vacate.v.02'),
 Synset('abandon.v.04'),
 Synset('abandon.v.05'),
 Synset('abandoned.s.01'),
 Synset('abandoned.s.02')]